In [67]:
import numpy as np 
from dotenv import load_dotenv
import os 
import json
import matplotlib.pyplot as plt
from statsmodels.stats.contingency_tables import mcnemar 
import csv

load_dotenv(override=True)

True

In [68]:
fcm_result_txt = f"{os.getenv('ROOT_PATH')}/results/fcm_result.txt"
lfcm_result_txt = f"{os.getenv('ROOT_PATH')}/results/lfcm_result.txt"

In [69]:
def get_prediction(logits):
    logits = np.array(logits)

    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=0)

    # Convert probabilities to predictions
    predictions = np.argmax(probabilities)

    return predictions 

In [70]:
def get_confusion_matrix(predicted_values, actual_values):
    confusion_matrix = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}
    if len(predicted_values) == len(actual_values):
        for i in range(len(predicted_values)):
            # TP
            if predicted_values[i] == 1 and actual_values[i] == 1:
                confusion_matrix['tp'] += 1
            #  TN
            elif predicted_values[i] == 0 and actual_values[i] == 0:
                confusion_matrix['tn'] += 1
            #  FP
            elif predicted_values[i] == 1 and actual_values[i] == 0:
                confusion_matrix['fp'] += 1
            #  FN
            elif predicted_values[i] == 0 and actual_values[i] == 1:
                confusion_matrix['fn'] += 1
        
        return confusion_matrix
    else:
        return None

In [71]:
def get_benchmark_scores(confusion_matrix):
    tp = confusion_matrix["tp"]
    tn = confusion_matrix["tn"]
    fp = confusion_matrix["fp"]
    fn = confusion_matrix["fn"]
    benchmark_scores = {"precision": 0, "recall": 0, "f1-score": 0, "accuracy": 0, 'specificity': 0}

    # precision
    benchmark_scores["precision"] = tp / (tp + fp)
    # recall
    benchmark_scores["recall"] = tp / (tp + fn)
    # f1-score
    benchmark_scores["f1-score"] = (
        2 * benchmark_scores["precision"] * benchmark_scores["recall"]
    ) / (benchmark_scores["precision"] + benchmark_scores["recall"])
    # accuracy
    benchmark_scores['accuracy'] = (tp + tn) / (tp + fn + tn + fp)
    # specificity
    benchmark_scores['specificity'] = tn / (tn + fp)

    return benchmark_scores

In [72]:
def get_mcnemar_contingency_table(a_predicted_values, b_predicted_values, actual_values):
    mcnemar_contigency_table = {"a": 0, "b": 0, "c": 0, "d": 0}
    for i in range(len(actual_values)):
        if (
            actual_values[i] == a_predicted_values[i]
            and actual_values[i] == b_predicted_values[i]
        ):
            mcnemar_contigency_table["a"] += 1
        elif (
            actual_values[i] == a_predicted_values[i]
            and actual_values[i] != b_predicted_values[i]
        ):
            mcnemar_contigency_table["b"] += 1
        elif (
            actual_values[i] != a_predicted_values[i]
            and actual_values[i] == b_predicted_values[i]
        ):
            mcnemar_contigency_table["c"] += 1
        elif (
            actual_values[i] != a_predicted_values[i]
            and actual_values[i] != b_predicted_values[i]
        ):
            mcnemar_contigency_table["d"] += 1
    return mcnemar_contigency_table
        

In [73]:
total = 0
correct = 0
confusion_matrix_fcm = dict()
model_a_actual_values = []
model_a_predicted_values = []
with open(fcm_result_txt, 'r', encoding='utf-8') as txt_file:
    predicted_values = []
    actual_values = []
    for f in txt_file:
        arr = f.split(',')
        logits = [float(arr[1]), float(arr[2])]
        target = int(arr[0])
        total += 1

        predicted_values.append(get_prediction(logits))
        actual_values.append(target)

        # if get_prediction(logits) == target:
        #     correct += 1
    
    confusion_matrix_fcm = get_confusion_matrix(predicted_values, actual_values)
    benchmark_score = get_benchmark_scores(confusion_matrix_fcm)
    model_a_actual_values = actual_values
    model_a_predicted_values = predicted_values

    print(confusion_matrix_fcm)
    print(benchmark_score)

{'tp': 357, 'tn': 266, 'fp': 682, 'fn': 111}
{'precision': 0.343599615014437, 'recall': 0.7628205128205128, 'f1-score': 0.4737889847378899, 'accuracy': 0.4399717514124294, 'specificity': 0.2805907172995781}


In [74]:
total = 0
correct = 0
confusion_matrix_lfcm = dict()
model_b_actual_values = []
model_b_predicted_values = []
with open(lfcm_result_txt, 'r', encoding='utf-8') as txt_file:
    predicted_values = []
    actual_values = []
    for f in txt_file:
        arr = f.split(',')
        logits = [float(arr[1]), float(arr[2])]
        target = int(arr[0])
        total += 1

        predicted_values.append(get_prediction(logits))
        actual_values.append(target)

        # if get_prediction(logits) == target:
        #     correct += 1
    
    confusion_matrix_lfcm = get_confusion_matrix(predicted_values, actual_values)
    benchmark_score = get_benchmark_scores(confusion_matrix_lfcm)
    model_b_actual_values = actual_values
    model_b_predicted_values = predicted_values

    print(confusion_matrix_lfcm)
    print(benchmark_score)

{'tp': 130, 'tn': 695, 'fp': 253, 'fn': 338}
{'precision': 0.3394255874673629, 'recall': 0.2777777777777778, 'f1-score': 0.30552291421856637, 'accuracy': 0.5826271186440678, 'specificity': 0.7331223628691983}


In [75]:
if (model_a_actual_values == model_b_actual_values) and (
    len(model_a_predicted_values) == len(model_b_predicted_values)
):
    mcnemar_contingency_table = get_mcnemar_contingency_table(
        model_a_predicted_values, model_b_predicted_values, model_a_actual_values
    )
    mc_table_arr = [
        [mcnemar_contingency_table["a"], mcnemar_contingency_table["b"]],
        [mcnemar_contingency_table["c"], mcnemar_contingency_table["d"]],
    ]

In [76]:
def get_tweet_ids(csv_file):
    tweet_ids = []
    with open(csv_file, 'r', encoding='utf-8') as csv_file:
        for f in csv_file:
            tweet_id = f.split(',')[0]
            tweet_ids.append(tweet_id)
    return tweet_ids[1:]

In [77]:
def interpret_label(label_str):
    if (label_str == '0'):
        return 'not racist'
    elif (label_str == '1'):
        return 'racist'
    else: return 'None'

In [78]:
def create_table_5(
    tweet_ids=[],
    actual_values=[],
    a_predicted_values=[],
    b_predicted_values=[],
    filepath="",
):
    with open(filepath, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['tweet no.', 'tweet filename', 'expected output', 'fcm output', 'lfcm output'])
        for i in range(len(tweet_ids)):
            a_predicted_value = interpret_label(str(a_predicted_values[i]))
            b_predicted_value = interpret_label(str(b_predicted_values[i]))
            actual_value = interpret_label(str(actual_values[i]))

            row = [
                i+1,
                f"{tweet_ids[i]}.json",
                actual_value,
                a_predicted_value,
                b_predicted_value,
            ]
            writer.writerow(row)

In [79]:
csv_filepath = f"{os.getenv('ROOT_PATH')}/test.csv"
table_5_filepath = f"{os.getenv('RESULTS_PATH')}/table5.csv"
tweet_ids = get_tweet_ids(csv_filepath)
actual_values = model_a_actual_values
a_pred_values = model_a_predicted_values
b_pred_values = model_b_predicted_values

create_table_5(tweet_ids, actual_values, a_pred_values, b_pred_values, table_5_filepath)

In [80]:
def create_table_6(
    confusion_matrix_model_a=dict(), confusion_matrix_model_b=dict(), filepath=""
):
    with open(filepath, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(
            ["model name", "TP", "TN", "FP", "FN", "Pre", "Rec", "F", "Acc"]
        )

        a_matrix = confusion_matrix_model_a
        b_matrix = confusion_matrix_model_b

        model_a_benchmark = get_benchmark_scores(confusion_matrix_model_a)
        model_b_benchmark = get_benchmark_scores(confusion_matrix_model_b)

        # model b
        writer.writerow(
            [
                b_matrix["name"],
                b_matrix["tp"],
                b_matrix["tn"],
                b_matrix["fp"],
                b_matrix["fn"],
                model_b_benchmark["precision"],
                model_b_benchmark["recall"],
                model_b_benchmark["f1-score"],
                model_b_benchmark["accuracy"],
            ]
        )

        # model a
        writer.writerow(
            [
                a_matrix["name"],
                a_matrix["tp"],
                a_matrix["tn"],
                a_matrix["fp"],
                a_matrix["fn"],
                model_a_benchmark["precision"],
                model_a_benchmark["recall"],
                model_a_benchmark["f1-score"],
                model_a_benchmark["accuracy"],
            ]
        )

In [81]:
model_a_confusion_matrix = confusion_matrix_fcm
model_a_confusion_matrix['name'] = 'FCM'

model_b_confusion_matrix = confusion_matrix_lfcm
model_b_confusion_matrix['name'] = 'LFCM'

table_6_filepath = f"{os.getenv('RESULTS_PATH')}/table6.csv"

create_table_6(model_a_confusion_matrix, model_b_confusion_matrix, table_6_filepath)

In [82]:
def create_table_7(mc_table, filepath=""):
    with open(filepath, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["A", "B", "C", "D", "McNemar Test Stats", "P-Value"])

        _mc_table_arr = [[mc_table["a"], mc_table["b"]], [mc_table["c"], mc_table["d"]]]
        _mcnemar = mcnemar(_mc_table_arr, exact=True)

        print(_mcnemar)
        writer.writerow([mc_table["a"], mc_table["b"], mc_table["c"], mc_table["d"], _mcnemar.statistic, float(str(_mcnemar.pvalue))])

In [83]:
table_7_filepath = f"{os.getenv('RESULTS_PATH')}/table7.csv"

create_table_7(mcnemar_contingency_table, table_7_filepath)

pvalue      7.644251120860157e-14
statistic   265.0


In [114]:
c_pred_values = []

for i in range(len(a_pred_values)):
    if a_pred_values[i] == 1 and b_pred_values[i] == 1:
        c_pred_values.append(b_pred_values[i])
    elif a_pred_values[i] == 0 and b_pred_values[i] == 0:
        c_pred_values.append(b_pred_values[i])
    elif b_pred_values[i] == 1 and a_pred_values[i] != 1:
        c_pred_values.append(a_pred_values[i])
    else:
        c_pred_values.append(b_pred_values[i])



confusion_matrix_c = get_confusion_matrix(c_pred_values, actual_values)
benchmark_score_c = get_benchmark_scores(confusion_matrix_c)

print(confusion_matrix_c)
print(benchmark_score_c)


{'tp': 120, 'tn': 723, 'fp': 225, 'fn': 348}
{'precision': 0.34782608695652173, 'recall': 0.2564102564102564, 'f1-score': 0.2952029520295203, 'accuracy': 0.5953389830508474, 'specificity': 0.7626582278481012}
